In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [7]:
import gseapy
import pandas as pd
from gprofiler import GProfiler


metabric_data = pd.read_csv('../data/metabric.csv.gz', low_memory=False)
print('Success!')
expression_data = metabric_data.iloc[27:,:]
print(expression_data)

new_clinical_patient = pd.read_csv(
    '../data/data_clinical_patient.txt', sep='\t', index_col=0).iloc[4:]
new_clinical_sample = pd.read_csv('../data/data_clinical_sample.txt', sep='\t', index_col=0).iloc[4:]
new_clinical = pd.concat([new_clinical_patient, new_clinical_sample.reindex(new_clinical_patient.index)], axis=1)
real_gene_names = pd.read_csv('../data/illumina2symbol.txt', sep="\t", index_col=0)

dtypedict = {}
for i in expression_data.columns[1:]:
    dtypedict[i] = 'str'
expression_data = expression_data.astype(dtypedict)

genes = expression_data['Unnamed: 0'].values.tolist()

gp = GProfiler(return_dataframe=True)
gp = gp.convert(organism='hsapiens',query=genes)

gp = gp.loc[gp['n_converted'] == 1]
gp = gp.loc[gp['name'] != 'None']
gp = gp.set_index('incoming')
gprofiler_names = gp


expression_data.index = expression_data['Unnamed: 0']
expression_data = expression_data.iloc[:, 1:]
expression_data = expression_data.rename(index=lambda x: gprofiler_names.loc[x, 'name'] if x in gprofiler_names.index else x)
expression_data = expression_data.rename(index=lambda x: real_gene_names.loc[x, 'symbol'] if x in real_gene_names.index else x)




Success!
         Unnamed: 0            MB-0002             MB-0008  \
27     ILMN_1802380   9.01387632374539    8.05012660652797   
28     ILMN_1893287   5.41685458556584    5.43417416873302   
29     ILMN_1736104   5.30362674026384   5.436291637861189   
30     ILMN_1792389  5.748716893322349    5.53058208099833   
31     ILMN_1854015   5.66622056832095    5.86469327569862   
...             ...                ...                 ...   
48825  ILMN_1852347   5.37278037640934    5.33931106739938   
48826  ILMN_1909210   5.27464373371713  5.3419149766887495   
48827  ILMN_1693941  6.938684625839911   6.272568454883519   
48828  ILMN_1846115     5.231857399442     5.2046295820244   
48829  ILMN_1709472   5.60455974703369    5.03063632170624   

                  MB-0010            MB-0035           MB-0036  \
27       8.38571533625692  9.613113232193239  9.39461874714461   
28       5.39564773719013   5.20667426697821  5.46867261874286   
29      5.670387473743251   5.36425576221325  5.

In [14]:
gseapatients = [x for x in expression_data.columns.tolist() if x in new_clinical.index]
new_clinical_patients = new_clinical.loc[gseapatients]


,Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Integrative Cluster,...,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage
#Patient Identifier,,,,,,,,,,,,,,,,,,,,,
MB-0002,0,4.02,High,NO,1,Positve,NEUTRAL,YES,Pre,4ER+,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,10,1
MB-0008,8,6.08,High,YES,1,Positve,NEUTRAL,YES,Post,9,...,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Positive,Negative,3,MDLC,Positive,Primary,40,2
MB-0010,0,4.062,Moderate,NO,1,Positve,NEUTRAL,YES,Post,7,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,31,4
MB-0035,0,3.056,High,NO,1,Negative,LOSS,NO,Post,3,...,Breast Cancer,Breast Invasive Lobular Carcinoma,Positive,Negative,2,ILC,Negative,Primary,28,2
MB-0036,0,3.044,Moderate,NO,1,Positve,NEUTRAL,YES,Post,3,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,2,IDC,Positive,Primary,22,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-7295,1,5.05,High,NO,4,Positve,NEUTRAL,YES,Pre,3,...,Breast Cancer,Breast Invasive Lobular Carcinoma,Positive,Negative,3,ILC,Positive,Primary,25,NaN
MB-7296,1,5.04,High,NO,4,Positve,GAIN,NO,Pre,5,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Positive,3,IDC,Negative,Primary,20,NaN
MB-7297,45,6.05,High,NO,4,Positve,NEUTRAL,YES,Post,1,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,25,NaN


In [15]:
new_clinical_patients = new_clinical.loc[gseapatients]
new_clinical_patients = new_clinical_patients.iloc[:,9]
gsea_list = new_clinical_patients.tolist()

In [16]:
gseapatients = []
for i in expression_data.columns.tolist():
    if i in new_clinical.index:
        gseapatients.append(i)
gseapatients
new_clinical_patients = new_clinical.loc[gseapatients]

In [17]:
new_clinical_patients

,Lymph nodes examined positive,Nottingham prognostic index,Cellularity,Chemotherapy,Cohort,ER status measured by IHC,HER2 status measured by SNP6,Hormone Therapy,Inferred Menopausal State,Integrative Cluster,...,Cancer Type,Cancer Type Detailed,ER Status,HER2 Status,Neoplasm Histologic Grade,Oncotree Code,PR Status,Sample Type,Tumor Size,Tumor Stage
#Patient Identifier,,,,,,,,,,,,,,,,,,,,,
MB-0002,0,4.02,High,NO,1,Positve,NEUTRAL,YES,Pre,4ER+,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,10,1
MB-0008,8,6.08,High,YES,1,Positve,NEUTRAL,YES,Post,9,...,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Positive,Negative,3,MDLC,Positive,Primary,40,2
MB-0010,0,4.062,Moderate,NO,1,Positve,NEUTRAL,YES,Post,7,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,31,4
MB-0035,0,3.056,High,NO,1,Negative,LOSS,NO,Post,3,...,Breast Cancer,Breast Invasive Lobular Carcinoma,Positive,Negative,2,ILC,Negative,Primary,28,2
MB-0036,0,3.044,Moderate,NO,1,Positve,NEUTRAL,YES,Post,3,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,2,IDC,Positive,Primary,22,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-7295,1,5.05,High,NO,4,Positve,NEUTRAL,YES,Pre,3,...,Breast Cancer,Breast Invasive Lobular Carcinoma,Positive,Negative,3,ILC,Positive,Primary,25,NaN
MB-7296,1,5.04,High,NO,4,Positve,GAIN,NO,Pre,5,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Positive,3,IDC,Negative,Primary,20,NaN
MB-7297,45,6.05,High,NO,4,Positve,NEUTRAL,YES,Post,1,...,Breast Cancer,Breast Invasive Ductal Carcinoma,Positive,Negative,3,IDC,Positive,Primary,25,NaN


In [23]:
def GSEA(receptor, level):
    classes = ['positive' if x == level else 'negative' for x in new_clinical_patients[receptor]]
    destination_path = f'GSEA_/{receptor}/'

    gs_res = gseapy.gsea(data=expression_data,
                         gene_sets='Reactome_2016',
                         method="signal_to_noise",
                         permutation_num=1000,
                         max_size=10000,
                         min_size=2,
                         processes=6,
                         cls=classes,
                         verbose=True,
                         outdir=destination_path)


GSEA('ER Status', 'Positive')


2020-03-26 14:31:56,959 Parsing data files for GSEA.............................
2020-03-26 14:31:59,516 Warning: dropping duplicated gene names, only keep the first values


KeyError: 'positive'